In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 64kB/s 
     |████████████████████████████████| 204kB 46.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=14498bef0249ed48f5f5ad5e85d92a58ac1ba14ddfd3a2589d53d5e053b1c90f
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('ML project').getOrCreate()

In [ ]:
df = spark.read.csv('bank.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [ ]:
class_name = df.columns[len(df.columns)-1]
df.groupBy(class_name).count().toPandas()

,deposit,count
0,no,5873
1,yes,5289


In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()

df1 = df

for index in range(len(df1.columns)):
  new_name = df.columns[index]+'_'
  indexer.setInputCol(df.columns[index]).setOutputCol(new_name)
  df1 = indexer.fit(df1).transform(df1)

In [ ]:
df1.show()

+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+----+----+--------+----------+--------+--------+--------+-----+--------+----+------+---------+---------+------+---------+---------+--------+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|age_|job_|marital_|education_|default_|balance_|housing_|loan_|contact_|day_|month_|duration_|campaign_|pdays_|previous_|poutcome_|deposit_|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+----+----+--------+----------+--------+--------+--------+-----+--------+----+------+---------+---------+------+---------+---------+--------+
| 59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|31.0| 3.0|     0.0|       0.0|     0.0|  2746

In [ ]:
class_name = 'deposit_'
features_names = df1.columns[17:-1]
print(features_names)

['age_', 'job_', 'marital_', 'education_', 'default_', 'balance_', 'housing_', 'loan_', 'contact_', 'day_', 'month_', 'duration_', 'campaign_', 'pdays_', 'previous_', 'poutcome_']


In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler()
assembler.setInputCols(features_names).setOutputCol('features')
transformed_data = assembler.transform(df1)

In [ ]:
transformed_data.show()

+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+----+----+--------+----------+--------+--------+--------+-----+--------+----+------+---------+---------+------+---------+---------+--------+--------------------+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|age_|job_|marital_|education_|default_|balance_|housing_|loan_|contact_|day_|month_|duration_|campaign_|pdays_|previous_|poutcome_|deposit_|            features|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+----+----+--------+----------+--------+--------+--------+-----+--------+----+------+---------+---------+------+---------+---------+--------+--------------------+
| 59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0

In [ ]:
#split data
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])

In [ ]:
from pyspark.ml.classification import LogisticRegression
model = LogisticRegression(featuresCol = 'features',labelCol=class_name, maxIter=30)

In [ ]:
M = model.fit(training_data)

In [ ]:
# Predict with the test dataset
predictions = M.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'deposit_', metricName = 'accuracy')
print('Logistic Regression Accuracy:', multi_evaluator.evaluate(predictions))

Logistic Regression Accuracy: 0.7892857142857143
